In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
project_id = "perceptive-ivy-290216"
df_bq = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

In [3]:
GP=df_bq[((df_bq["GP"]=="Australian Grand Prix") & (df_bq["Year"]==2024))]
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
GP.head()

<ipython-input-3-18c3450404b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GP["LapNumber"]=GP["LapNumber"].astype(int)


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
108838,0 days 01:00:05.328000,ALB,23,0 days 00:01:35.752000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.858000,0 days 00:00:38.140000,NaT,0 days 00:59:27.287000,0 days 01:00:05.463000,265.0,310.0,292.0,238.0,False,MEDIUM,1.0,True,Williams,0 days 00:58:29.232000,NaT,1,14.0,False,,False,False,2024,Australian Grand Prix
107952,0 days 01:01:29.864000,ALB,23,0 days 00:01:24.536000,2,1.0,NaT,NaT,0 days 00:00:30.021000,0 days 00:00:18.135000,0 days 00:00:36.380000,0 days 01:00:35.294000,0 days 01:00:53.429000,0 days 01:01:29.809000,273.0,321.0,306.0,279.0,True,MEDIUM,2.0,True,Williams,0 days 01:00:05.328000,NaT,1,14.0,False,,False,True,2024,Australian Grand Prix
107953,0 days 01:02:53.470000,ALB,23,0 days 00:01:23.606000,3,1.0,NaT,NaT,0 days 00:00:29.307000,0 days 00:00:18.047000,0 days 00:00:36.252000,0 days 01:01:59.116000,0 days 01:02:17.163000,0 days 01:02:53.415000,269.0,319.0,305.0,312.0,True,MEDIUM,3.0,True,Williams,0 days 01:01:29.864000,NaT,1,14.0,False,,False,True,2024,Australian Grand Prix
107954,0 days 01:04:16.908000,ALB,23,0 days 00:01:23.438000,4,1.0,NaT,NaT,0 days 00:00:29.061000,0 days 00:00:18.076000,0 days 00:00:36.301000,0 days 01:03:22.476000,0 days 01:03:40.552000,0 days 01:04:16.853000,272.0,322.0,304.0,NaN,True,MEDIUM,4.0,True,Williams,0 days 01:02:53.470000,NaT,1,13.0,False,,False,True,2024,Australian Grand Prix
108384,0 days 01:05:40.824000,ALB,23,0 days 00:01:23.916000,5,1.0,NaT,NaT,0 days 00:00:29.328000,0 days 00:00:18.079000,0 days 00:00:36.509000,0 days 01:04:46.181000,0 days 01:05:04.260000,0 days 01:05:40.769000,275.0,325.0,306.0,308.0,False,MEDIUM,5.0,True,Williams,0 days 01:04:16.908000,NaT,1,13.0,False,,False,True,2024,Australian Grand Prix


In [23]:
fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
#             markers=True,
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=1000, width=1800, template="plotly_dark",
         color_discrete_map={
                 "VER": "#3671C6",
                 "PER": "#3671C6",
                 "HAM": "#6CD3BF",
                 "RUS": "#6CD3BF",
                 "PIA": "#F58020",
                 "NOR": "#F58020",
                 "SAI": "#F91536",
                 "LEC": "#F91536",
                 "ALO": "#358C75",
                 "STR": "#358C75",
                 "GAS": "#2293D1",
                 "OCO": "#2293D1",
                 "ALB": "#37BEDD",
                 "SAR": "#37BEDD",
                 "TSU": "#5E8FAA",
                 "RIC": "#5E8FAA",
                 "DEV": "#5E8FAA",
                 "LAW": "#5E8FAA",
                 "BOT": "#C92D4B",
                 "ZHO": "#C92D4B",
                 "MAG": "#B6BABD",
                 "HUL": "#B6BABD",

         },
        )
fig.update_yaxes(autorange="reversed")

fig.update_layout(
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 2
    ),

    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),

    margin_pad=10
)

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": t.name,
                    "method": "restyle",
                    "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
                }
                for t in fig.data
            ]
        }
    ]
)

In [ ]:
fig.write_html("ladder.html")

In [5]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

In [10]:
fig=px.scatter(GP, x="LapNumber", y="LapTime (s)", color='Driver',
#             markers=True,
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=1000, width=1800, template="plotly_dark",
         color_discrete_map={
                 "VER": "#3671C6",
                 "PER": "#3671C6",
                 "HAM": "#6CD3BF",
                 "RUS": "#6CD3BF",
                 "PIA": "#F58020",
                 "NOR": "#F58020",
                 "SAI": "#F91536",
                 "LEC": "#F91536",
                 "ALO": "#358C75",
                 "STR": "#358C75",
                 "GAS": "#2293D1",
                 "OCO": "#2293D1",
                 "ALB": "#37BEDD",
                 "SAR": "#37BEDD",
                 "TSU": "#5E8FAA",
                 "RIC": "#5E8FAA",
                 "DEV": "#5E8FAA",
                 "LAW": "#5E8FAA",
                 "BOT": "#C92D4B",
                 "ZHO": "#C92D4B",
                 "MAG": "#B6BABD",
                 "HUL": "#B6BABD",

         },
        )
fig.update_yaxes(autorange="reversed")

fig.update_layout(
    font=dict(
        family="Roboto",
        size=14,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 2
    ),

    margin_pad=10
)

In [7]:
GP=GP.sort_values(['LapNumber', "Driver"])
GP.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime_TD,LapTime (s)
108838,0 days 01:00:05.328000,ALB,23,00:01:35.752000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.858000,0 days 00:00:38.140000,NaT,0 days 00:59:27.287000,0 days 01:00:05.463000,265.0,310.0,292.0,238.0,False,MEDIUM,1.0,True,Williams,0 days 00:58:29.232000,NaT,1,14.0,False,,False,False,2024,Australian Grand Prix,0 days 00:01:35.752000,95.752
108833,0 days 01:00:04.079000,ALO,14,00:01:34.503000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.774000,0 days 00:00:37.212000,NaT,0 days 00:59:26.965000,0 days 01:00:04.137000,259.0,292.0,285.0,233.0,False,HARD,2.0,False,Aston Martin,0 days 00:58:29.232000,NaT,1,11.0,False,,False,False,2024,Australian Grand Prix,0 days 00:01:34.503000,94.503
108847,0 days 01:00:04.339000,BOT,77,00:01:34.763000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.782000,0 days 00:00:37.741000,NaT,0 days 00:59:26.567000,0 days 01:00:04.427000,270.0,302.0,286.0,257.0,False,MEDIUM,2.0,False,Kick Sauber,0 days 00:58:29.232000,NaT,1,12.0,False,,False,False,2024,Australian Grand Prix,0 days 00:01:34.763000,94.763
108831,0 days 01:00:06.880000,GAS,10,00:01:37.304000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.854000,0 days 00:00:38.564000,NaT,0 days 00:59:28.406000,0 days 01:00:06.942000,267.0,316.0,287.0,223.0,False,MEDIUM,1.0,True,Alpine,0 days 00:58:29.232000,NaT,1,17.0,False,,False,False,2024,Australian Grand Prix,0 days 00:01:37.304000,97.304
108844,0 days 01:00:03.264000,HAM,44,00:01:33.688000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.674000,0 days 00:00:36.983000,NaT,0 days 00:59:26.244000,0 days 01:00:03.352000,266.0,311.0,290.0,231.0,False,SOFT,1.0,True,Mercedes,0 days 00:58:29.232000,NaT,1,10.0,False,,False,False,2024,Australian Grand Prix,0 days 00:01:33.688000,93.688


In [9]:
fig2=px.scatter(GP,
        x="LapNumber",
        y="Position",
        color='DriverNumber',
        animation_frame="LapNumber",
        # animation_group='LapNumber',
        range_x=[0,65],
        text='Driver',
        range_y=[0,20],
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=1000, width=1800, template="plotly_dark",
        color_discrete_map={
                 1: "#3671C6",
                 11: "#3671C6",
                 44: "#6CD3BF",
                 63: "#6CD3BF",
                 81: "#F58020",
                 4: "#F58020",
                 55: "#F91536",
                 16: "#F91536",
                 14: "#358C75",
                 18: "#358C75",
                 10: "#2293D1",
                 31: "#2293D1",
                 23: "#37BEDD",
                 2: "#37BEDD",
                 22: "#5E8FAA",
                 3: "#5E8FAA",
                 21: "#5E8FAA",
                 40: "#5E8FAA",
                 77: "#C92D4B",
                 24: "#C92D4B",
                 20: "#B6BABD",
                 27: "#B6BABD"
         },
         )


fig2.update_yaxes(autorange="reversed")

fig.update_traces(textposition='top center')

fig2.update_layout(
    font=dict(
        family="Roboto",
        size=14,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 1
    ),

    margin_pad=10
)

# for x,y in zip(GP[GP["Driver"]=="SAI"].Driver, GP[GP["Driver"]=="SAI"].Position):
#   for png in (Path.cwd().joinpath("f1").glob("*.png")):
#     if str.split(str.split(str(png),".")[0],"/")[3]==x:
#       image=str(png)
#       fig2.add_layout_image(
#           x=x,
#           y=y+4,
#           source=Image.open(image),
#           xref="x",
#           yref="y",
#           sizex=5,
#           sizey=5,
#           xanchor="center",
#           yanchor="middle",
#       )
fig2.show()

In [ ]:
fig2.write_html("ladder2.html")

# Fastest Driver in each lap

In [ ]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["RK"] = GP_Rk.groupby("LapNumber")["LapTime"].rank(method="dense", ascending=True)
GP_FASTEST_LAPS=GP_Rk[GP_Rk["RK"]==1.0].sort_values("LapNumber")
GP_FASTEST_LAPS=GP_FASTEST_LAPS[["Time","Driver","Team","LapTime","LapNumber","Stint","PitOutTime","PitInTime","Sector1Time","Sector2Time","Sector3Time","SpeedI1","SpeedI2","TyreLife","TrackStatus","Position","Year","GP","RK"]]
GP_FASTEST_LAPS

,Time,Driver,Team,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,TyreLife,TrackStatus,Position,Year,GP,RK
106172,0 days 00:59:57.034000,VER,Red Bull Racing,00:01:27.458000,1,1.0,NaT,NaT,NaT,0 days 00:00:18.462000,0 days 00:00:35.518000,278.0,285.0,1.0,1,1.0,2024,Australian Grand Prix,1.0
105371,0 days 01:01:20.551000,SAI,Ferrari,00:01:22.600000,2,1.0,NaT,NaT,0 days 00:00:29.006000,0 days 00:00:18.026000,0 days 00:00:35.568000,277.0,320.0,2.0,1,1.0,2024,Australian Grand Prix,1.0
105355,0 days 01:02:45.199000,NOR,McLaren,00:01:22.656000,3,1.0,NaT,NaT,0 days 00:00:28.750000,0 days 00:00:18.204000,0 days 00:00:35.702000,271.0,299.0,3.0,1,3.0,2024,Australian Grand Prix,1.0
105372,0 days 01:04:05.990000,SAI,Ferrari,00:01:22.433000,4,1.0,NaT,NaT,0 days 00:00:28.643000,0 days 00:00:18.497000,0 days 00:00:35.293000,270.0,291.0,4.0,1,1.0,2024,Australian Grand Prix,1.0
106130,0 days 01:05:31.688000,PIA,McLaren,00:01:22.594000,5,1.0,NaT,NaT,0 days 00:00:28.691000,0 days 00:00:17.918000,0 days 00:00:35.985000,NaN,324.0,5.0,1,4.0,2024,Australian Grand Prix,1.0
106003,0 days 01:06:51.191000,SAI,Ferrari,00:01:22.572000,6,1.0,NaT,NaT,0 days 00:00:28.648000,0 days 00:00:18.453000,0 days 00:00:35.471000,NaN,293.0,6.0,1,1.0,2024,Australian Grand Prix,1.0
106004,0 days 01:08:13.625000,SAI,Ferrari,00:01:22.434000,7,1.0,NaT,NaT,0 days 00:00:28.670000,0 days 00:00:18.462000,0 days 00:00:35.302000,272.0,295.0,7.0,1,1.0,2024,Australian Grand Prix,1.0
105373,0 days 01:09:35.928000,SAI,Ferrari,00:01:22.303000,8,1.0,NaT,NaT,0 days 00:00:28.562000,0 days 00:00:18.397000,0 days 00:00:35.344000,274.0,296.0,8.0,1,1.0,2024,Australian Grand Prix,1.0
105374,0 days 01:10:57.953000,SAI,Ferrari,00:01:22.025000,9,1.0,NaT,NaT,0 days 00:00:28.594000,0 days 00:00:18.290000,0 days 00:00:35.141000,273.0,295.0,9.0,1,1.0,2024,Australian Grand Prix,1.0
105375,0 days 01:12:19.668000,SAI,Ferrari,00:01:21.715000,10,1.0,NaT,NaT,0 days 00:00:28.426000,0 days 00:00:18.281000,0 days 00:00:35.008000,274.0,295.0,10.0,1,1.0,2024,Australian Grand Prix,1.0
